# VacationPy
----

#### Note
The weather data was retrieved on 07/19/2020 14:00pm (Pacific Time) via http://api.openweathermap.org

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [105]:
df=pd.read_csv("output_data/cities.csv")
#df=df.iloc[1:150, 0:6]
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arraial do Cabo,-22.97,-42.03,73.40,78,0,17.22,BR,1595192182
1,1,Mar del Plata,-38.00,-57.56,55.40,93,75,13.87,AR,1595192184
2,2,Trairi,-3.28,-39.27,77.77,74,23,11.07,BR,1595192186
3,3,George Town,5.41,100.34,80.60,100,40,4.70,MY,1595192098
4,4,Avarua,-21.21,-159.78,75.20,83,83,16.11,CK,1595192189


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [110]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [111]:
# Store latitude and longitude in locations
loc = df[["Lat", "Lng"]]

#making a list to input in the map
humidity_list =df["Humidity"]

In [130]:
# Plot a map
fig = gmaps.figure(center=(0.0,-0.0),zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(loc,weights=humidity_list,max_intensity=100)
# Add layer to the figure
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [131]:
#set criteria to filter out the cities with the best weather
temp_df=df.loc[df["Max Temp"]<90,["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
hum_df=temp_df.loc[temp_df["Humidity"]<20,:]
cloud_df=hum_df.loc[hum_df["Cloudiness"]<10,:]
final_df=cloud_df.loc[cloud_df["Wind Speed"]<15,:]
final_df=final_df.reset_index(drop=True)
final_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Isa,-20.73,139.50,60.80,19,7,13.80,AU,1595192367
1,Bam,29.11,58.36,82.74,15,0,11.88,IR,1595192403
2,Nurota,40.56,65.69,84.20,16,0,4.70,UZ,1595192518
3,Ardakān,32.31,54.02,86.00,16,0,6.93,IR,1595192564
4,Sangīn,32.07,64.84,81.79,15,0,7.07,AF,1595192654
5,Baker City,44.77,-117.83,89.60,19,1,4.70,US,1595192990
6,Fasa,28.94,53.65,86.00,16,0,9.17,IR,1595193077


In [132]:
#set the map frame
fig = gmaps.figure(center=(30.0,-0.0),zoom_level=2)
# Create heat layer of humidity
heat_layer = gmaps.heatmap_layer(loc, weights=humidity_list, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.5)
# Add heat layer to the figure
fig.add_layer(heat_layer)


coordinates=final_df[['Lat','Lng']]
# Assign the marker layer to a variable and drop the coordinate on the map
markers = gmaps.marker_layer(coordinates)
# Add the marker layer to the map
fig.add_layer(markers)

#show figfure
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map with Info Boxes
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [95]:
# base url for Google places
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#fixed parameters
target_radius = 5000
target_type = "hotel"

hotel_names=[]

for i in range(len(final_df)):
    # geocoordinates
    target_coordinates = f"{str(final_df['Lat'].values[i])}, {str(final_df['Lng'].values[i])}"
    # set up a parameters dictionary. The parameters name are totally Google's call
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
     }
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    hotel_names.append(response['results'][1]['name'])

In [133]:
#add a new column to the "final" data frame to create hotle dataframe
hotel_df=final_df.assign(HotelName=hotel_names)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
0,Mount Isa,-20.73,139.50,60.80,19,7,13.80,AU,1595192367,ibis Styles Mt Isa Verona
1,Bam,29.11,58.36,82.74,15,0,11.88,IR,1595192403,شرکت بازرگانی مهد بازرگان جنوب
2,Nurota,40.56,65.69,84.20,16,0,4.70,UZ,1595192518,Guest House Ruslan Nurata & Tours
3,Ardakān,32.31,54.02,86.00,16,0,6.93,IR,1595192564,Mellat Bank
4,Sangīn,32.07,64.84,81.79,15,0,7.07,AF,1595192654,Myanroday
5,Baker City,44.77,-117.83,89.60,19,1,4.70,US,1595192990,Bridge Street Inn
6,Fasa,28.94,53.65,86.00,16,0,9.17,IR,1595193077,Azin Cabin


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [134]:
#set the map frame
fig = gmaps.figure(center=(30.0,-0.0),zoom_level=2)
# Create heat layer of humidity
heat_layer = gmaps.heatmap_layer(loc, weights=humidity_list, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.5)
# Add heat layer to the figure
fig.add_layer(heat_layer)

#assign markers with info boxes
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))